In [1]:
import openmm.app as app
import openmm as mm
import openmm.unit as unit
import numpy as np
import jax
import jax.numpy as jnp
import dmff
from dmff.api.xmlio import XMLIO
from dmff.api.paramset import ParamSet
from dmff.generators.classical import CoulombGenerator, LennardJonesGenerator
from dmff.api.hamiltonian import Hamiltonian
from dmff.operators import ParmedLennardJonesOperator
from dmff import NeighborListFreud
from dmff.mbar import ReweightEstimator
import mdtraj as md
from tqdm import tqdm, trange
import parmed
import sys
import os

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.


In [2]:
prm_top = parmed.load_file("Lig.top")
prm_top_500 = prm_top * 500
dmfftop = dmff.DMFFTopology(from_top=prm_top_500.topology)
prmop = ParmedLennardJonesOperator()
dmfftop = prmop(dmfftop, gmx_top = prm_top_500)

prmop.renderLennardJonesXML("init.xml")
xmlio = XMLIO()
xmlio.loadXML("init.xml")
ffinfo = xmlio.parseXML()
cov_mat = dmfftop.buildCovMat()

paramset = ParamSet()
lj_gen = LennardJonesGenerator(ffinfo, paramset)

In [3]:
lj_force = lj_gen.createPotential(
    dmfftop, nonbondedMethod=app.CutoffPeriodic, nonbondedCutoff=1.0, args={})

### 基于给定拓扑进行采样的函数

In [4]:
def runMD(topfile, pdbfile, trajfile, length):
    try:
        os.remove("Lig_500.top")
    except:
        pass
    top_prm = parmed.load_file(topfile)
    top_500 = top_prm * 500
    top_500.save("Lig_500.top")
    pdb = app.PDBFile(pdbfile)
    top = app.GromacsTopFile("Lig_500.top")
    top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
    system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)
    for force in system.getForces():
        if isinstance(force, mm.NonbondedForce):
            force.setUseDispersionCorrection(False)
    system.addForce(mm.MonteCarloBarostat(1.0*unit.bar, 300*unit.kelvin, 25))
    integ = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 2.5*unit.femtosecond)
    simulation = app.Simulation(top.topology, system, integ)
    simulation.reporters.append(app.StateDataReporter(sys.stdout, 400, time=True, potentialEnergy=True, temperature=True, density=True, speed=True, remainingTime=True, totalSteps=int(length) * 400))
    simulation.reporters.append(app.DCDReporter(trajfile, 400))
    simulation.context.setPositions(pdb.getPositions())
    simulation.minimizeEnergy(maxIterations=200)
    simulation.step(int(length) * 400)
    os.remove("Lig_500.top")

# runMD("Lig.top", "init.pdb", "traj.dcd", 100)

In [5]:
def rerun_energy(pdb, traj, top, skip=20, removeLJ=True):
    samples = md.load(traj, top=pdb)
    try:
        os.remove("Lig_500.top")
    except:
        pass
    top_prm = parmed.load_file(top)
    top_500 = top_prm * 500
    top_500.save("Lig_500.top")
    pdb = app.PDBFile(pdb)
    top = app.GromacsTopFile("Lig_500.top")
    os.remove("Lig_500.top")
    top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
    system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)
    
    for force in system.getForces():
        if isinstance(force, mm.NonbondedForce):
            force.setUseDispersionCorrection(False)
            if removeLJ:
                for npart in range(force.getNumParticles()):
                    chrg, sig, eps = force.getParticleParameters(npart)
                    force.setParticleParameters(npart, chrg, 1.0, 0.0)
                for nex in range(force.getNumExceptions()):
                    p1, p2, chrg, sig, eps = force.getExceptionParameters(nex)
                    force.setExceptionParameters(nex, p1, p2, chrg, 1.0, 0.0)
    integ = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 2.5*unit.femtosecond)
    ctx = mm.Context(system, integ)
    energies = []
    for frame in tqdm(samples):
        ctx.setPositions(frame.xyz[0] * unit.nanometer)
        ctx.setPeriodicBoxVectors(*frame.unitcell_vectors[0])
        ctx.applyConstraints(1e-10)
        state = ctx.getState(getEnergy=True)
        energy = state.getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
        energies.append(energy)
    return np.array(energies)

# rerun_energy("init.pdb", "traj.dcd", "Lig.top")

In [29]:
def rerun_dmff_lennard_jones(params, pdb, traj, efunc, skip=0):
    samples = md.load(traj, top=pdb)[skip:]
    energies = []
    nblist = NeighborListFreud(samples.unitcell_vectors[0], 1.0, cov_mat)
    xyzs_jnp = jnp.array(samples.xyz)
    cell_jnp = jnp.array(samples.unitcell_vectors)
    energies = []
    nblist = NeighborListFreud(samples.unitcell_vectors[0], 1.0, cov_mat)
    xyzs_jnp = jnp.array(samples.xyz)
    cell_jnp = jnp.array(samples.unitcell_vectors)
    energies = []
    for nframe, frame in tqdm(enumerate(samples)):
        # calc pair
        pairs = jnp.array(nblist.allocate(frame.xyz[0], frame.unitcell_vectors[0]))
        ener = efunc(xyzs_jnp[nframe,:,:], cell_jnp[nframe,:,:], pairs, params)
        energies.append(ener.reshape((1,)))
    energies = jnp.concatenate(energies)
    return energies

In [7]:
os.system("cp Lig.top loop-0.top")
for nloop in range(1, 51):
    # sample
    print("SAMPLE")
    runMD(f"loop-{nloop-1}.top", "init.pdb", f"loop-{nloop}.dcd", length=250)
    print("RERUN")
    ener = rerun_energy("init.pdb", f"loop-{nloop}.dcd", f"loop-{nloop-1}.top", removeLJ=False)
    ener_no_lj = rerun_energy("init.pdb", f"loop-{nloop}.dcd", f"loop-{nloop-1}.top")
    print("ESTIMATOR")
    traj = md.load(f"loop-{nloop}.dcd", top="init.pdb")
    estimator = ReweightEstimator(ener[50:], base_energies=ener_no_lj[50:], volume=traj.unitcell_volumes[50:])
    estimator.optimize_mbar()

    print("CALC DENSE")
    density = md.density(traj[50:]) * 0.001

    # get loss & grad
    def loss(paramset):
        lj_jax = rerun_dmff_lennard_jones(paramset, "init.pdb", f"loop-1.dcd", lj_force, skip=50)
        weight = estimator.estimate_weight(lj_jax)
        dens = weight * density
        dens = dens.mean()
        return jnp.power(dens - 0.85, 2)
    
    v, g = jax.value_and_grad(loss, 0)(paramset)
    # update parameters
    break

SAMPLE
#"Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Density (g/mL)","Speed (ns/day)","Time Remaining"
0.9999999999999897,9222.664181707602,196.16601595835834,1.0708585680505172,0,--
1.9999999999999685,15798.195431707602,260.68737021094995,1.0543216270012699,527,0:40
2.999999999999947,17640.6485567076,287.97812576612154,1.039247896379932,530,0:40
3.999999999999926,18860.1798067076,287.7537716135091,1.0233154451817923,533,0:39
5.000000000000082,19898.0313692076,295.3900253358711,1.0192671840659897,521,0:40
6.000000000000238,20142.1719942076,301.4870722412037,1.02370288062679,523,0:40
7.000000000000394,19701.2188692076,301.68303708095436,1.0243677262841633,524,0:40
8.00000000000055,19861.1485567076,304.8092362035773,1.0192882259511267,524,0:39
9.000000000000352,20235.4532442076,303.4533370272648,1.0229632773553843,526,0:39
10.000000000000153,20332.0938692076,303.81937694477125,1.0229949634522584,526,0:39
10.999999999999954,21024.2032442076,299.42708092248023,1.020708581908

100%|██████████| 250/250 [00:08<00:00, 31.18it/s]


ESTIMATOR
CALC DENSE


In [7]:
ener = rerun_energy("init.pdb", f"loop-1.dcd", f"loop-0.top", removeLJ=False)
ener_no_lj = rerun_energy("init.pdb", f"loop-1.dcd", f"loop-0.top")

traj = md.load(f"loop-1.dcd", top="init.pdb")

100%|██████████| 250/250 [00:08<00:00, 28.63it/s]


In [30]:
estimator = ReweightEstimator(ener[50:], base_energies=ener_no_lj[50:], volume=traj.unitcell_volumes[50:])

density = md.density(traj[50:]) * 0.001

def loss(paramset):
    lj_jax = rerun_dmff_lennard_jones(paramset, "init.pdb", f"loop-1.dcd", lj_force, skip=50)
    weight = estimator.estimate_weight(lj_jax)
    dens = weight * density
    dens = dens.mean()
    return jnp.power(dens - 0.85, 2)

v, g = jax.value_and_grad(loss, 0)(paramset)

200it [00:53,  3.73it/s]


In [32]:
print(v)
print(g.parameters)

0.03009764
{'LennardJonesForce': {'epsilon': Array([ 1.6125038e-01,  1.5715137e-02,  2.7147807e-02,  3.9004363e-02,
       -1.1221571e-03,  8.3593066e+12], dtype=float32), 'epsilon_nbfix': Array([], dtype=float32), 'sigma': Array([-2.4031186e-01, -8.6094892e-01,  3.4750976e-02, -1.7451453e-01,
       -7.4366671e-01,  5.4654611e-18], dtype=float32), 'sigma_nbfix': Array([], dtype=float32)}}
